In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flights-dataset/flights_df1.csv
/kaggle/input/flights-dataset/flights_df.csv


In [2]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
OG_df = pd.read_csv('/kaggle/input/flights-dataset/flights_df.csv')

In [4]:
OG_df.shape

(1877274, 30)

In [5]:
df = OG_df[OG_df['ArrDel15'] == 1.0]

In [6]:
df.head()

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,...,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
7,2016-01-01,1,2016,1,1,949.0,1.0,924,25.0,11057,...,1023,92,42,10,46,43,90,2016-01-01,900.0,11057
11,2016-01-01,1,2016,1,1,957.0,1.0,930,27.0,11057,...,1023,92,42,10,46,43,90,2016-01-01,900.0,11057
24,2016-01-01,1,2016,1,1,1446.0,1.0,1325,81.0,11057,...,1023,91,41,8,49,47,76,2016-01-01,1300.0,11057
46,2016-01-02,1,2016,1,2,957.0,1.0,929,28.0,11057,...,1025,0,34,6,40,37,82,2016-01-02,900.0,11057
47,2016-01-02,1,2016,1,2,948.0,1.0,929,19.0,11057,...,1025,0,34,6,40,37,82,2016-01-02,900.0,11057


In [7]:
df.shape

(388058, 30)

In [8]:
df.isnull().sum()

FlightDate         0
Quarter            0
Year               0
Month              0
DayofMonth         0
DepTime            0
DepDel15           0
CRSDepTime         0
DepDelayMinutes    0
OriginAirportID    0
DestAirportID      0
ArrTime            0
CRSArrTime         0
ArrDel15           0
ArrDelayMinutes    0
windspeedKmph      0
winddirDegree      0
weatherCode        0
precipMM           0
visibility         0
pressure           0
cloudcover         0
DewPointF          0
WindGustKmph       0
tempF              0
WindChillF         0
humidity           0
date               0
time               0
airport            0
dtype: int64

## Splitting Data

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(['ArrDel15','ArrTime', 'ArrDelayMinutes', 'Quarter', 'date', 'airport', 'FlightDate'], axis = 1), df['ArrDelayMinutes'], test_size = 0.2)

In [10]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(310446, 23) (310446,)
(77612, 23) (77612,)


In [11]:
from sklearn.preprocessing import scale
X_train  = scale(X_train)
X_test = scale(X_test)

## Linear Regression

In [12]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [13]:
lr = lr.fit(X_train, Y_train)

In [14]:
y_preds_lr = lr.predict(X_test)

In [15]:
print("MAE: " + str(mean_absolute_error(Y_test, y_preds_lr)))
print("MSE: " + str(mean_squared_error(Y_test, y_preds_lr)))
print("R2: " + str(r2_score(Y_test, y_preds_lr)))

MAE: 12.427485989726534
MSE: 318.37967920913735
R2: 0.935930569232742


## Extra Trees Regressor

In [16]:
from sklearn.ensemble import ExtraTreesRegressor
etr = ExtraTreesRegressor(n_estimators = 100)

In [17]:
etr = etr.fit(X_train, Y_train)

In [18]:
y_preds_et = etr.predict(X_test)

In [19]:
print("MAE: " + str(mean_absolute_error(Y_test, y_preds_et)))
print("MSE: " + str(mean_squared_error(Y_test, y_preds_et)))
print("R2: " + str(r2_score(Y_test, y_preds_et)))

MAE: 12.099680204092149
MSE: 293.2383966719064
R2: 0.9409898985998688


## Gradient Boosting Regressor

In [20]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()

In [21]:
gbr = gbr.fit(X_train, Y_train)

In [22]:
y_preds_gb = gbr.predict(X_test)

In [23]:
print("MAE: " + str(mean_absolute_error(Y_test, y_preds_gb)))
print("MSE: " + str(mean_squared_error(Y_test, y_preds_gb)))
print("R2: " + str(r2_score(Y_test, y_preds_gb)))

MAE: 11.865933497011595
MSE: 294.17833890252183
R2: 0.9408007484511557


## Pipeline

In [24]:
OG_df.dropna(inplace = True)

In [25]:
from sklearn.preprocessing import scale
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import ExtraTreesClassifier

In [26]:
X = OG_df.drop(['ArrDel15','ArrTime', 'ArrDelayMinutes', 'Quarter', 'date', 'airport', 'FlightDate'], axis = 1)
Y = OG_df[['ArrDelayMinutes', 'ArrDel15']]

In [27]:
X_train_p, X_test_p, Y_train_p, Y_test_p = train_test_split(X, Y.drop(['ArrDelayMinutes'], axis = 1), test_size = 0.2)

X_train_p = X_train_p.reset_index()
X_test_p = X_test_p.reset_index()

X_train_p = scale(X_train_p)
X_test_p = scale(X_test_p)

In [28]:
sm = SMOTE()
X_train_ps, Y_train_ps = sm.fit_sample(X_train_p, Y_train_p)

In [29]:
etc = ExtraTreesClassifier(n_estimators = 100)
etc.fit(X_train_ps, Y_train_ps)
y_preds_p = etc.predict(X_test_p)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [30]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_p, y_preds_p))

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94    292535
         1.0       0.77      0.77      0.77     77751

    accuracy                           0.90    370286
   macro avg       0.85      0.85      0.85    370286
weighted avg       0.90      0.90      0.90    370286



In [31]:
X_tr_p, X_t_p, Y_tr_p, Y_t_p = train_test_split(X, Y, test_size = 0.2)

In [32]:
train_truth_val = Y_tr_p['ArrDel15'] == 1

In [33]:
X_train_pf = X_tr_p[train_truth_val]
Y_train_pf = Y_tr_p[train_truth_val]['ArrDelayMinutes']

In [34]:
print(len(Y_train_pf), len(X_train_pf))

310534 310534


In [35]:
len(y_preds_p)

370286

In [36]:
test_truth_val = y_preds_p == 1

In [37]:
X_test_pf = X_t_p[test_truth_val]
Y_test_pf = Y_t_p[test_truth_val]['ArrDelayMinutes']

In [38]:
len(X_test_pf), len(Y_test_pf)

(77664, 77664)

#### Linear Regression

In [39]:
lr1 = LinearRegression()
lr1.fit(X_train_pf, Y_train_pf)
y_preds_plr = lr1.predict(X_test_pf)

In [40]:
print("MAE: " + str(mean_absolute_error(Y_test_pf, y_preds_plr)))
print("MSE: " + str(mean_squared_error(Y_test_pf, y_preds_plr)))
print("R2: " + str(r2_score(Y_test_pf, y_preds_plr)))

MAE: 20.946192137108035
MSE: 509.1596445331863
R2: 0.7144976112128003


#### Extra Trees Regressor

In [41]:
etr1 = ExtraTreesRegressor(n_estimators = 100)
etr1.fit(X_train_pf, Y_train_pf)
y_preds_pet = etr1.predict(X_test_pf)

In [42]:
print("MAE: " + str(mean_absolute_error(Y_test_pf, y_preds_pet)))
print("MSE: " + str(mean_squared_error(Y_test_pf, y_preds_pet)))
print("R2: " + str(r2_score(Y_test_pf, y_preds_pet)))

MAE: 22.65166795426452
MSE: 601.809038681371
R2: 0.6625460796391518


#### Gradient Boosting

In [43]:
gbr1 = GradientBoostingRegressor()
gbr1.fit(X_train_pf, Y_train_pf)
y_preds_pgb = gbr1.predict(X_test_pf)

In [44]:
print("MAE: " + str(mean_absolute_error(Y_test_pf, y_preds_pgb)))
print("MSE: " + str(mean_squared_error(Y_test_pf, y_preds_pgb)))
print("R2: " + str(r2_score(Y_test_pf, y_preds_pgb)))

MAE: 22.592000815443814
MSE: 583.5251601370452
R2: 0.6727984455851724


## Regression Testing

In [45]:
truth_val_1 = Y_test >= 15; truth_val_2 = Y_test < 100
tv1 = [i&j for i,j in zip(truth_val_1, truth_val_2)]

truth_val_1 = Y_test >= 100; truth_val_2 = Y_test < 200
tv2 = [i&j for i,j in zip(truth_val_1, truth_val_2)]

truth_val_1 = Y_test >= 200; truth_val_2 = Y_test < 500
tv3 = [i&j for i,j in zip(truth_val_1, truth_val_2)]

truth_val_1 = Y_test >= 500; truth_val_2 = Y_test < 1000
tv4 = [i&j for i,j in zip(truth_val_1, truth_val_2)]

truth_val_1 = Y_test >= 1000
tv5 = truth_val_1

In [46]:
X1 = X_test[tv1]
Y1 = Y_test[tv1]

X2 = X_test[tv2]
Y2 = Y_test[tv2]

X3 = X_test[tv3]
Y3 = Y_test[tv3]

X4 = X_test[tv4]
Y4 = Y_test[tv4]

X5 = X_test[tv5]
Y5 = Y_test[tv5]

### Slot1 15-100

In [47]:
y_preds_slot1 = etr.predict(X1)
print("MAE: " + str(mean_absolute_error(Y1, y_preds_slot1)))
print("MSE: " + str(mean_squared_error(Y1, y_preds_slot1)))
print("R2: " + str(r2_score(Y1, y_preds_slot1)))

MAE: 10.737502312031815
MSE: 199.80472177433026
R2: 0.591783038720278


### Slot 2 100-200

In [48]:
y_preds_slot2 = etr.predict(X2)
print("MAE: " + str(mean_absolute_error(Y2, y_preds_slot2)))
print("MSE: " + str(mean_squared_error(Y2, y_preds_slot2)))
print("R2: " + str(r2_score(Y2, y_preds_slot2)))

MAE: 18.610859987618657
MSE: 717.4430580195005
R2: 0.01356092871820358


### Slot 3 200-500

In [49]:
y_preds_slot3 = etr.predict(X3)
print("MAE: " + str(mean_absolute_error(Y3, y_preds_slot3)))
print("MSE: " + str(mean_squared_error(Y3, y_preds_slot3)))
print("R2: " + str(r2_score(Y3, y_preds_slot3)))

MAE: 20.03369401648155
MSE: 920.2403162307417
R2: 0.7843170117389957


### Slot 4 500 - 1000

In [50]:
y_preds_slot4 = etr.predict(X4)
print("MAE: " + str(mean_absolute_error(Y4, y_preds_slot4)))
print("MSE: " + str(mean_squared_error(Y4, y_preds_slot4)))
print("R2: " + str(r2_score(Y4, y_preds_slot4)))

MAE: 23.739863013698628
MSE: 1084.759598630137
R2: 0.9436713517709263


### Slot 5 1000+

In [51]:
y_preds_slot5 = etr.predict(X5)
print("MAE: " + str(mean_absolute_error(Y5, y_preds_slot5)))
print("MSE: " + str(mean_squared_error(Y5, y_preds_slot5)))
print("R2: " + str(r2_score(Y5, y_preds_slot5)))

MAE: 30.09937500000001
MSE: 1139.8909437500004
R2: 0.926870573140057
